# Scrape

For the scrapping part I'll use Beatiful Soap.


In [54]:
# libraries to import
import pandas as pd
import numpy as np
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup

Necessary steps to convert a html page into a dataframe.

In [57]:
# scrape the data from the webpage to a table with 3 columns
PageToScrape = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
PageToScrape = BeautifulSoup(PageToScrape, 'lxml')
PageToScrape = PageToScrape.find('table',{'class':'wikitable sortable'})
PageToScrape = PageToScrape.find_all('tr')
#PageToScrape

table = []

for row in PageToScrape:
    table.append([t.text.strip() for t in row.find_all('td')])

    
df = pd.DataFrame(table, columns=['PostalCode', 'Borough', 'Neighbourhood'])
print(df.head(5))
print(df.shape)

  PostalCode       Borough     Neighbourhood
0       None          None              None
1        M1A  Not assigned      Not assigned
2        M2A  Not assigned      Not assigned
3        M3A    North York         Parkwoods
4        M4A    North York  Victoria Village
(181, 3)


After crating the table I will clean the table, removing nulls and other invalid data.

In [58]:
# cleaning the table
df1 = df
df1 = df1[~df1['PostalCode'].isnull()]  
df1.drop(df1[df1['Borough']=="Not assigned"].index,axis=0, inplace=True)
print(df1.head(5))
print(df1.shape)

  PostalCode           Borough                                Neighbourhood
3        M3A        North York                                    Parkwoods
4        M4A        North York                             Victoria Village
5        M5A  Downtown Toronto                    Regent Park, Harbourfront
6        M6A        North York             Lawrence Manor, Lawrence Heights
7        M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government
(103, 3)


In [59]:
# merging postal codes
df2 = df1
df2 = df2.reset_index()
df2= df1.groupby('PostalCode').agg(lambda x: ','.join(x))
df2.loc[df2['Neighbourhood']=="Not assigned",'Neighbourhood']=df2.loc[df2['Neighbourhood']=="Not assigned",'Borough']
print(df2.head(5))
print(df2.shape)

                Borough                           Neighbourhood
PostalCode                                                     
M1B         Scarborough                          Malvern, Rouge
M1C         Scarborough  Rouge Hill, Port Union, Highland Creek
M1E         Scarborough       Guildwood, Morningside, West Hill
M1G         Scarborough                                  Woburn
M1H         Scarborough                               Cedarbrae
(103, 2)


In [63]:
# Final table
df3 = df2
df3 = df3.reset_index()
print(df3.head(15))
print(df3.shape)

   PostalCode      Borough                                      Neighbourhood
0         M1B  Scarborough                                     Malvern, Rouge
1         M1C  Scarborough             Rouge Hill, Port Union, Highland Creek
2         M1E  Scarborough                  Guildwood, Morningside, West Hill
3         M1G  Scarborough                                             Woburn
4         M1H  Scarborough                                          Cedarbrae
5         M1J  Scarborough                                Scarborough Village
6         M1K  Scarborough        Kennedy Park, Ionview, East Birchmount Park
7         M1L  Scarborough                    Golden Mile, Clairlea, Oakridge
8         M1M  Scarborough    Cliffside, Cliffcrest, Scarborough Village West
9         M1N  Scarborough                        Birch Cliff, Cliffside West
10        M1P  Scarborough  Dorset Park, Wexford Heights, Scarborough Town...
11        M1R  Scarborough                                  Wexf